# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('homepage_actions.csv')

# Check unique values in the 'id' column
unique_ids = data['id'].unique()

# Find the number of viewers and clickers
viewers = data[data['action'] == 'view']['id'].unique()
clickers = data[data['action'] == 'click']['id'].unique()
num_viewers = len(viewers)
num_clickers = len(clickers)

# Number of viewers who also clicked
viewers_and_clickers = set(viewers) & set(clickers)
num_viewers_and_clickers = len(viewers_and_clickers)

# Identify anomalies - individuals who clicked without viewing
anomalies = set(clickers) - set(viewers)
num_anomalies = len(anomalies)

# Check for overlap between control and experiment groups
overlap = set(data[data['group'] == 'control']['id']).intersection(set(data[data['group'] == 'experiment']['id']))
num_overlap = len(overlap)

# Print results
print(data.head())

print(f"Number of viewers: {num_viewers} ,  number of clickers:{num_clickers} " )
print(f"Number of viewers who also clicked: {num_viewers_and_clickers}")
print(f"Number of anomalies (clicked without viewing): {num_anomalies}")
print(f"Number of overlapping IDs between control and experiment groups: {num_overlap}")

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Number of viewers: 6328 ,  number of clickers:1860 
Number of viewers who also clicked: 1860
Number of anomalies (clicked without viewing): 0
Number of overlapping IDs between control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
from scipy.stats import ttest_ind

# Filter data for clicks only
click_data = data[data['action'] == 'click']

# Separate data into control and experiment groups
control_ids = click_data[click_data['group'] == 'control']['id']
experiment_ids = click_data[click_data['group'] == 'experiment']['id']

# Check if there is variability in the data
if control_ids.nunique() == 1 or experiment_ids.nunique() == 1:
    print("Not enough variability in the data to perform t-test.")
else:
    # Perform independent samples t-test
    t_stat, p_value = ttest_ind(control_ids, experiment_ids, equal_var=False)

    # Print the results
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")

    # Interpret the results
    alpha = 0.05  # significance level
    if p_value < alpha:
        print("The experimental homepage is considered more effective than the control group.")
    else:
        print("There is no significant difference between the experimental and control groups.")

T-statistic: -0.5827851039715325
P-value: 0.5601087216132623
There is no significant difference between the experimental and control groups.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
#Your code here
# Expected number of clicks for the experiment group
expected_experiment_clicks = len(control_ids) #control_ctr * experiment_ids.nunique()

print(f"Expected number of clicks for the experiment group: {expected_experiment_clicks}")

Expected number of clicks for the experiment group: 932


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
#Your code here
from scipy.stats import norm

# Calculate the actual number of clicks for the experiment group
actual_experiment_clicks = len(experiment_ids)
# Calculate the standard deviation of the binomial distribution
binomial_std =  (1 - (actual_experiment_clicks - expected_experiment_clicks)) **0.5 # Assuming a binomial distribution
# Calculate the z-score
z_score = (actual_experiment_clicks - expected_experiment_clicks) / binomial_std
z_score

-1.7888543819998317

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here

# Calculate the two-tailed p-value
p_value = 2 * (1 - norm.cdf(abs(z_score)))

print(f"P-value based on z-score: {p_value}")

P-value based on z-score: 0.07363827012030266


### Analysis:

Does this result roughly match that of the previous statistical test?

The values are different due to the nature of the tests, based on a significance level (e.g., 0.05). However, as observed, the calculated p-values are greater than 0.05, and you would generally fail to reject the null hypothesis in both cases. Both p-values are above my significance level of 0.05. The results are roughly consistent in terms of failing to reject the null hypothesis that "There is no significant difference between the experimental and control groups".


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.